In [1]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn import set_config

In [2]:
set_config(transform_output='pandas')

In [3]:
df = pd.read_csv('data/insurance.csv')
df.head()

age  sex  children  smoker      charges
0   19    1         0       1  16884.92400
1   18    0         1       0   1725.55230
2   28    0         3       0   4449.46200
3   33    0         0       0  21984.47061
4   32    0         0       0   3866.85520

In [4]:
#Question 1
np.random.seed(2025)

train_df, test_df = train_test_split(df, test_size=0.25)

print(f'length of training data:{len(train_df)}')
print(f'length of test data:{len(test_df)}')

length of training data:1003
length of test data:335


In [5]:
#Question 2
mse_result = {'selected predictor':[], 'mean_squared_error':[],'best_K':[]}

names = ['age', 'sex', 'children', 'smoker']
n_totals = len(names)

selected = []
col_names = []
model_0 = KNeighborsRegressor()
scaler = StandardScaler()
train_df_new = train_df.copy()
test_df_new = test_df.copy()
train_df_new[['age','children']] = scaler.fit_transform(train_df[['age','children']])
test_df_new[['age','children']] = scaler.transform(test_df[['age','children']])

param_grid = {'n_neighbors':range(1,61,5)}
knn_grid = GridSearchCV(
    estimator=model_0,
    param_grid=param_grid,
    cv=5,
    scoring = "neg_root_mean_squared_error"
)
for i in range(1, n_totals+1):
    mse = np.zeros(len(names))
    k_best = np.zeros(len(names))
    for j in range(len(names)):
        X = train_df_new[selected + [names[j]]]
        y = train_df_new['charges']
        knn_grid.fit(X, y)
        mse_grid = pd.DataFrame(knn_grid.cv_results_)
        best_idx = mse_grid['mean_test_score'].argmax()
        mse[j] = mse_grid.loc[best_idx,'mean_test_score']
        k_best[j] = mse_grid.loc[best_idx,'param_n_neighbors']
    best_set = selected + [names[mse.argmax()]]
    
    mse_result['selected predictor'].append(','.join(best_set))
    mse_result['mean_squared_error'].append(mse.max())
    mse_result['best_K'].append(k_best[mse.argmax()])
    selected = best_set
    del names[mse.argmax()]

mse_df = pd.DataFrame(mse_result)
mse_df
        

selected predictor  mean_squared_error  best_K
0                   smoker        -7567.640357    51.0
1               smoker,age        -6409.314841    41.0
2      smoker,age,children        -6492.028049    26.0
3  smoker,age,children,sex        -6633.836098    16.0

In [6]:
#Question3
# the selected predictors: smoker, age  
# the number of K: 41
model_1 = KNeighborsRegressor(n_neighbors=41)
model_1.fit(train_df_new[['smoker','age']],train_df_new['charges'])
test_df_new['prediction'] = model_1.predict(test_df_new[['smoker','age']])
PMSPE = mean_squared_error(
    y_true = test_df_new['charges'],
    y_pred = test_df_new['prediction']
)**(1/2)
PMSPE

6625.50888385552

In [7]:
#Question4
result = {'selected predictor':[],'PMSPE':[]}

names = ['age', 'sex', 'children', 'smoker']
n_totals = len(names)

selected = []

model_2 = LinearRegression()
param_grid = {}
lm_grid = GridSearchCV(
    estimator=model_2,
    param_grid=param_grid,
    cv=5,
    scoring = 'neg_root_mean_squared_error'
)

for i in range(1,n_totals+1):
    mse = np.zeros(len(names))
    for j in range(len(names)):
        X = train_df[selected + [names[j]]]
        y = train_df['charges']
        lm_grid.fit(X, y)
        mse[j] = lm_grid.best_score_           
    print(mse)
    best_set = selected + [names[mse.argmax()]]

    result['selected predictor'].append(','.join(best_set))
    result['PMSPE'].append(-(mse.max()))
    selected = best_set
    del names[mse.argmax()]

result_df = pd.DataFrame(result)
result_df

[-11571.48284194 -12185.59522701 -12176.38966262  -7531.52424667]
[-6358.44599836 -7550.89167478 -7495.9277851 ]
[-6375.21337042 -6335.67900637]
[-6351.3145783]


selected predictor        PMSPE
0                   smoker  7531.524247
1               smoker,age  6358.445998
2      smoker,age,children  6335.679006
3  smoker,age,children,sex  6351.314578

In [8]:
#Question5
model_3 = LinearRegression()
model_3.fit(train_df[['smoker','age','children']],train_df['charges'])
test_df_renew = test_df.copy()
test_df_renew['prediction'] = model_3.predict(test_df_renew[['smoker','age','children']])
test_df_renew
PMSPE = mean_squared_error(
    y_true = test_df_renew['charges'],
    y_pred = test_df_renew['prediction']
)**(1/2)
PMSPE

6465.287747419116

In [9]:
#Question6
print(f'intercept:{model_3.intercept_}')
print(f'coefficient:{model_3.coef_}')

intercept:-3226.60747771515
coefficient:[23957.93484107   282.00633358   521.25276392]
